In [1]:
import pandas as pd

In [3]:
model_file = "/Users/rezadoobary/Downloads/bayes_results_merged/20251207-013645_train_567891011_notest_vstep1/merged_looporder_20251207-013650/inference_model_sklearn.pkl"
feature_names = '/Users/rezadoobary/Downloads/bayes_results_merged/20251207-013645_train_567891011_notest_vstep1/merged_looporder_20251207-013650/feature_names.json'

In [4]:
for i in range(5,10):   
    print(i,pd.read_csv(f"/Users/rezadoobary/Documents/MLCORRELATORS/ML-correlator/Tree_classifier_for_graphs/new_stuff/features/merged/{i}loops.csv")['Basic_num_nodes'].unique())

5 [9]
6 [10]
7 [11]
8 [12]
9 [13]


In [5]:
import json
import pickle

# Load modela
with open(model_file, "rb") as f:
    model = pickle.load(f)

# Load feature names
with open(feature_names, "r") as f:
    feature_names = json.load(f)

print("Model loaded:", type(model))
print("Number of features:", len(feature_names))


Model loaded: <class 'xgboost.sklearn.XGBClassifier'>
Number of features: 98


/var/folders/4s/7f5zf8596_526pnhktn077f40000gn/T/ipykernel_10142/90445857.py:6: UserWarning: [16:49:07] WARNING: /Users/runner/work/xgboost/xgboost/src/gbm/../common/error_msg.h:82: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  model = pickle.load(f)


In [7]:
# test example


def correct_dataset(df):
       n = 16
       df["Centrality_closeness_mean_norm"] = df["Centrality_closeness_mean"] * (n - 1)
       df["Centrality_closeness_max_norm"]  = df["Centrality_closeness_max"]  * (n - 1)

       # Optionally handle rows where any inputs are NaN:
       mask =  ~np.isfinite(df["Centrality_closeness_mean"]) | \
              ~np.isfinite(df["Centrality_closeness_max"])

       df.loc[mask, ["Centrality_closeness_mean_norm", "Centrality_closeness_max_norm"]] = np.nan
       return df


In [8]:
from tqdm import tqdm
import numpy as np

def predict_with_progress(model, df, feature_names, batch_size=5000):
    X = df[feature_names]
    n = len(X)
    preds = []

    for start in tqdm(range(0, n, batch_size), desc="Predicting", unit="rows"):
        end = min(start + batch_size, n)
        batch = X.iloc[start:end]
        batch_pred = model.predict_proba(batch)[:, 1]
        preds.append(batch_pred)

    return np.concatenate(preds)

# Run with progress

scores_12loops = []
for i in range(1,21):
    df = pd.read_parquet(f"/Users/rezadoobary/Documents/MLCORRELATORS/ML-correlator/Tree_classifier_for_graphs/new_stuff/features/12loops/merged_data/{i}part.parquet")
    df = correct_dataset(df)
    predictions = predict_with_progress(model, df, feature_names)
    scores_12loops.extend(predictions)
    print(i, len(scores_12loops))


Predicting: 100%|██████████| 200/200 [00:01<00:00, 119.36rows/s]


1 1000000


Predicting: 100%|██████████| 200/200 [00:01<00:00, 121.37rows/s]


2 2000000


Predicting: 100%|██████████| 200/200 [00:01<00:00, 114.56rows/s]


...

18 18000000


Predicting: 100%|██████████| 200/200 [00:06<00:00, 28.80rows/s]


19 19000000


Predicting: 100%|██████████| 43/43 [00:00<00:00, 136.89rows/s]

20 19212867


In [ ]:
scores_12loops = [float(x) for x in scores_12loops]
pickle.dump(
    scores_12loops,
    open('/Users/rezadoobary/Documents/MLCORRELATORS/ML-correlator/Tree_classifier_for_graphs/new_stuff/results/scores/loop_scores_vs1.pkl', 'wb')
)

In [11]:
# loop order

In [12]:
from tqdm import tqdm
import numpy as np



coefficients = []
for i in range(1,21):
    df = pd.read_parquet(f"/Users/rezadoobary/Documents/MLCORRELATORS/ML-correlator/Tree_classifier_for_graphs/new_stuff/features/12loops/merged_data/{i}part.parquet")
    coefficients.extend(df['COEFFICIENTS'].values)

In [13]:
coefficients = [int(x) for x in coefficients]

In [14]:
pickle.dump(
    coefficients,
    open('/Users/rezadoobary/Documents/MLCORRELATORS/ML-correlator/Tree_classifier_for_graphs/new_stuff/results/scores/coefficients.pkl', 'wb')
)

In [15]:
from sklearn.metrics import roc_auc_score

In [17]:
coefficients = pickle.load(
    open('/Users/rezadoobary/Documents/MLCORRELATORS/ML-correlator/Tree_classifier_for_graphs/new_stuff/results/scores/coefficients.pkl', 'rb')
)

loop_scores = pickle.load(
    open('/Users/rezadoobary/Documents/MLCORRELATORS/ML-correlator/Tree_classifier_for_graphs/new_stuff/results/scores/loop_scores_vs1.pkl', 'rb')
)


In [18]:
roc_auc_score(coefficients, loop_scores)

0.9065544382829686

In [21]:
model_file = "/Users/rezadoobary/Downloads/bayes_results_merged/20251207-160836_train_567891011_notest_vstep12/merged_looporder_20251207-160841/inference_model_sklearn.pkl"
feature_names = '/Users/rezadoobary/Downloads/bayes_results_merged/20251207-160836_train_567891011_notest_vstep12/merged_looporder_20251207-160841/feature_names.json'

In [22]:
import json
import pickle

# Load modela
with open(model_file, "rb") as f:
    model = pickle.load(f)

# Load feature names
with open(feature_names, "r") as f:
    feature_names = json.load(f)

print("Model loaded:", type(model))
print("Number of features:", len(feature_names))


Model loaded: <class 'xgboost.sklearn.XGBClassifier'>
Number of features: 98


In [23]:
from tqdm import tqdm
import numpy as np

def predict_with_progress(model, df, feature_names, batch_size=5000):
    X = df[feature_names]
    n = len(X)
    preds = []

    for start in tqdm(range(0, n, batch_size), desc="Predicting", unit="rows"):
        end = min(start + batch_size, n)
        batch = X.iloc[start:end]
        batch_pred = model.predict_proba(batch)[:, 1]
        preds.append(batch_pred)

    return np.concatenate(preds)

# Run with progress

scores_12loops = []
for i in range(1,21):
    df = pd.read_parquet(f"/Users/rezadoobary/Documents/MLCORRELATORS/ML-correlator/Tree_classifier_for_graphs/new_stuff/features/12loops/merged_data/{i}part.parquet")
    df = correct_dataset(df)
    predictions = predict_with_progress(model, df, feature_names)
    scores_12loops.extend(predictions)
    print(i, len(scores_12loops))


Predicting: 100%|██████████| 200/200 [00:07<00:00, 27.52rows/s]


1 1000000


Predicting: 100%|██████████| 200/200 [00:04<00:00, 49.59rows/s]


2 2000000


Predicting: 100%|██████████| 200/200 [00:08<00:00, 23.80rows/s]


...

18 18000000


Predicting: 100%|██████████| 200/200 [00:03<00:00, 57.48rows/s]


19 19000000


Predicting: 100%|██████████| 43/43 [00:02<00:00, 18.84rows/s]

20 19212867


In [25]:
scores_12loops = [float(x) for x in scores_12loops]
pickle.dump(
    scores_12loops,
    open('/Users/rezadoobary/Documents/MLCORRELATORS/ML-correlator/Tree_classifier_for_graphs/new_stuff/results/scores/loop_scores_vs12.pkl', 'wb')
)

In [26]:
roc_auc_score(coefficients, scores_12loops)

0.9059547652450028

In [70]:
vals = np.array(np.array(strat_scores) > 0.29, dtype = int)

from sklearn.metrics import precision_score, recall_score, f1_score, balanced_accuracy_score
import pandas as pd

# Example: y_true and y_pred already defined
y_true = coefficients
y_pred = vals

metrics = {
    "Precision": precision_score(y_true, y_pred, zero_division=0),
    "Recall": recall_score(y_true, y_pred, zero_division=0),
    "F1 Score": f1_score(y_true, y_pred, zero_division=0),
    "Balanced Accuracy": balanced_accuracy_score(y_true, y_pred)
}

metrics_df = pd.DataFrame(metrics, index=["Score"]).T  # T = Transpose for column format
metrics_df


,Score
Precision,0.623746
Recall,0.689544
F1 Score,0.654997
Balanced Accuracy,0.774524


In [71]:
vals = np.array(np.array(loop_scores) > 0.38, dtype = int)

from sklearn.metrics import precision_score, recall_score, f1_score, balanced_accuracy_score
import pandas as pd

# Example: y_true and y_pred already defined
y_true = coefficients
y_pred = vals

metrics = {
    "Precision": precision_score(y_true, y_pred, zero_division=0),
    "Recall": recall_score(y_true, y_pred, zero_division=0),
    "F1 Score": f1_score(y_true, y_pred, zero_division=0),
    "Balanced Accuracy": balanced_accuracy_score(y_true, y_pred)
}

metrics_df = pd.DataFrame(metrics, index=["Score"]).T  # T = Transpose for column format
metrics_df


,Score
Precision,0.662899
Recall,0.737354
F1 Score,0.698147
Balanced Accuracy,0.805350


In [72]:
# quikc look at f-graphs

In [1]:
import pandas as pd
from fgraph_features_cli3 import build_graphs_from_row, extract_features_row, parse_groups

df = pd.read_csv("/Users/rezadoobary/Downloads/graph_data_5.csv")
row = df.iloc[0]

groups = parse_groups("all")

feats = extract_features_row(
    row,
    groups=groups,
    lap_eigs_k=10,
    netlsd_points=32,
    netlsd_tmin=0.01,
    netlsd_tmax=100.0,
    ind4_exact_nmax=50,
    ind4_max_samples=50000,
    ind5_sample_frac=1.0,
    distance_max_sources=200,
    distance_exact_if_leq=500,
    seed=42,
    intra_workers=0,
    betweenness_k=0,
)

for k in sorted([c for c in feats.keys() if not str(feats[c]) == "nan"])[:40]:
    print(k, "=", feats[k])

Adjacency_energy_DEN = 17.05997848692523
Adjacency_energy_NUM = 6.0
Adjacency_energy_TOTAL = 16.13930578120349
Adjacency_energy_over_fro_DEN = 2.6324118308137825
Adjacency_energy_over_fro_NUM = 2.4494897427831783
Adjacency_energy_over_fro_TOTAL = 2.3295081343278796
Adjacency_energy_per_node_DEN = 1.8955531652139141
Adjacency_energy_per_node_NUM = 1.0
Adjacency_energy_per_node_TOTAL = 1.793256197911499
Adjacency_estrada_index_DEN = 122.00314220011559
Adjacency_estrada_index_NUM = 9.258483808891462
Adjacency_estrada_index_TOTAL = 245.58968114293356
Adjacency_estrada_per_node_DEN = 13.555904688901732
Adjacency_estrada_per_node_NUM = 1.5430806348152437
Adjacency_estrada_per_node_TOTAL = 27.287742349214838
Adjacency_moment_2_DEN = 4.666666666666667
Adjacency_moment_2_NUM = 1.0
Adjacency_moment_2_TOTAL = 5.333333333333333
Adjacency_moment_2_over_avgdeg_DEN = 1.0
Adjacency_moment_2_over_avgdeg_NUM = 1.0
Adjacency_moment_2_over_avgdeg_TOTAL = 1.0
Adjacency_moment_3_DEN = 9.333333333333334
Adja

/opt/miniconda3/envs/torch-env/lib/python3.10/site-packages/tqdm_joblib/__init__.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [44]:
fgraph5 = pd.read_csv("/Users/rezadoobary/Documents/MLCORRELATORS/ML-correlator/Tree_classifier_for_graphs/new_stuff/features/fgraphs2/5_fgraph_feats.csv")

In [45]:
fgraph5['COEFFICIENTS']

0    1
1   -1
2    1
3    1
4   -1
5    1
6    1
Name: COEFFICIENTS, dtype: int64